In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "True"

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import librosa
from pydub import AudioSegment
import torchaudio
import torchaudio.functional as F
import torch.quantization as quantization
from pydub import AudioSegment
import IPython.display as play
import soundfile as sf

In [3]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

PyTorch version: 2.5.1
CUDA available: True
CUDA version: 11.8


In [4]:
from speechbrain.inference.separation import SepformerSeparation as separator
model = separator.from_hparams(source = "speechbrain/sepformer-wsj03mix", savedir = "pretrained_models/sepformer-wsj03mix")

INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [disable_jit_profiling, allow_tf32]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
INFO:speechbrain.utils.fetching:Fetch hyperparams.yaml: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj03mix' if not cached
c:\Users\ashut\anaconda3\Lib\site-packages\speechbrain\utils\fetching.py:151: UserWarning: Using SYMLINK strategy on Windows for fetching potentially requires elevated privileges and is not recommended. See `LocalStrategy` documentation.
  warnings.warn(
INFO:speechbrain.utils.fetching:Fetch custom.py: Fetching from HuggingFace Hub 'speechbrain/sepformer-wsj03mix' if not cached
c:\Users\ashut\anaconda3\Lib\site-packages\speechbrain\utils\parameter_transfer.py:234: UserWarning: Requested Pretrainer collection using symlinks on Windows. This might not work; see `LocalStrategy` documentation. Consider unsetting `collect_in` i

In [ ]:
ip = 'MixedSpeech.m4a'
op = 'MixedSpeech.wav'

audio = AudioSegment.from_file(ip,format='mp4')
audio.export(op,format = 'wav')

In [5]:
print("Available audio backends:", torchaudio.list_audio_backends())

Available audio backends: ['ffmpeg', 'soundfile']


In [6]:
torchaudio.set_audio_backend("soundfile")
waveform, sample_rate = torchaudio.load(r"C:\Users\ashut\SelectiveNoiseReduction_MiniProject_SEM6\MixedSpeech.wav")

resampled_waveform = F.resample(waveform, sample_rate, 8000)
torchaudio.save(r"C:\Users\ashut\SelectiveNoiseReduction_MiniProject_SEM6\MixedSpeech.wav", resampled_waveform, 8000)
print("File resampled to 8 kHz and saved as 'MixedSpeech.wav'")

File resampled to 8 kHz and saved as 'MixedSpeech.wav'


C:\Users\ashut\AppData\Local\Temp\ipykernel_12640\130636759.py:1: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [7]:
model.eval() #Set the model to evaluation mode
quantized_model = quantization.quantize_dynamic(
    model, #original model
    {torch.nn.Linear}, 
    dtype = torch.qint8 # Convert the floating point weights to 8-bit integer weights
)
torch.save(quantized_model.state_dict(),"quantized_sepformer.pth")
print("Model Quantized Successfully.")

with torch.no_grad():
    est_sources = quantized_model.separate_file(path = "MixedSpeech.wav")
torchaudio.save("SQ1.wav", est_sources[:, :, 0].detach().cpu(), 8000)
torchaudio.save("SQ2.wav", est_sources[:, :, 1].detach().cpu(), 8000)
torchaudio.save("SQ3.wav", est_sources[:, :, 2].detach().cpu(), 8000)

Model Quantized Successfully.


In [8]:
ip = 'SQ2_Reference.m4a'
op = 'SQ2_Reference.wav'

audio = AudioSegment.from_file(ip,format='mp4')
audio.export(op,format = 'wav')

print("Conversion Complete")

Conversion Complete


In [9]:
torchaudio.set_audio_backend("soundfile")
waveform, sample_rate = torchaudio.load(r"C:\Users\ashut\SelectiveNoiseReduction_MiniProject_SEM6\SQ2_Reference.wav")

resampled_waveform = F.resample(waveform, sample_rate, 8000)
torchaudio.save(r"C:\Users\ashut\SelectiveNoiseReduction_MiniProject_SEM6\SQ2_Reference.wav", resampled_waveform, 8000)
print("File resampled to 8 kHz and saved as 'SQ2_Reference.wav'")

File resampled to 8 kHz and saved as 'SQ2_Reference.wav'


C:\Users\ashut\AppData\Local\Temp\ipykernel_12640\4234511507.py:1: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


In [10]:
# Load the estimated and reference signals without resampling
y_est, sr_est = librosa.load("SQ2.wav", sr=None)
y_ref, sr_ref = librosa.load("SQ2_Reference.wav", sr=None)

# Ensure the sample rates match
assert sr_est == sr_ref, "Sample rates must be the same!"

# Pad SQ2_Reference to match SQ2
if len(y_ref) < len(y_est):
    pad_width = len(y_est) - len(y_ref)
    y_ref_padded = np.pad(y_ref, (0, pad_width), mode='constant')
else:
    y_ref_padded = y_ref[:len(y_est)]  # Just in case it’s longer

# Save the padded reference back to the same file
sf.write("SQ2_Reference.wav", y_ref_padded, sr_ref)

print(f"Padding complete: SQ2_Reference.wav now has shape {y_ref_padded.shape}")


c:\Users\ashut\anaconda3\Lib\inspect.py:1000: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__file__'):


Padding complete: SQ2_Reference.wav now has shape (91819,)


In [11]:
y,sr = librosa.load("SQ2.wav")
y.shape

(253077,)

In [13]:
x,sr = librosa.load("SQ2_Reference.wav")
x.shape

(253077,)

In [14]:
# Save the quantized model
quantized_model_path = "quantized_sepformer.pth"
torch.save(quantized_model.state_dict(), quantized_model_path)
print(f"Quantized model saved to {quantized_model_path}")

Quantized model saved to quantized_sepformer.pth


In [15]:
# Get the size of the quantized model file
file_size_bytes = os.path.getsize(quantized_model_path)
file_size_kb = file_size_bytes / 1024  # Convert to KB
file_size_mb = file_size_kb / 1024  # Convert to MB

print(f"Size of the quantized model: {file_size_mb:.2f} MB")

Size of the quantized model: 60.21 MB


In [16]:
def si_snr(reference, estimation, eps=1e-8):
    """
    Compute the Scale-Invariant Signal-to-Noise Ratio (SI-SNR) between a reference
    and an estimated signal.

    Parameters:
        reference (np.ndarray): Clean reference signal.
        estimation (np.ndarray): Estimated signal.
        eps (float): Small constant to avoid division by zero.

    Returns:
        float: SI-SNR value in decibels (dB).
    """
    # Remove the mean (DC component) from both signals
    reference = reference - np.mean(reference)
    estimation = estimation - np.mean(estimation)

    # Compute the scaling factor to align estimation to reference
    scaling_factor = np.dot(estimation, reference) / (np.linalg.norm(reference) ** 2 + eps)
    
    # Projection of the estimated signal on the reference
    s_target = scaling_factor * reference

    # Compute the noise (error) component
    e_noise = estimation - s_target

    # Calculate SI-SNR in dB
    si_snr_value = 10 * np.log10(np.sum(s_target ** 2) / (np.sum(e_noise ** 2) + eps))
    return si_snr_value

# Load the estimated and reference signals without resampling (assumes they are at the same rate)
y_est, sr_est = librosa.load("SQ2.wav", sr=None)
y_ref, sr_ref = librosa.load("SQ2_Reference.wav", sr=None)

# Ensure the sample rates match
assert sr_est == sr_ref, "Sample rates must be the same!"

# Match the signal lengths by trimming the longer one
min_length = min(len(y_est), len(y_ref))
y_est = y_est[:min_length]
y_ref = y_ref[:min_length]

# Calculate SI-SNR
score = si_snr(y_ref, y_est)
print("SI-SNR: {:.2f} dB".format(score))

SI-SNR: -47.20 dB


In [17]:
play.Audio("MixedSpeech.wav")

In [18]:
play.Audio("SQ2_Reference.wav")

In [19]:
play.Audio("SQ2.wav")

In [20]:
play.Audio("SQ3.wav")